# Level 1: Processing raw IREM data

In [14]:
import os
from pathlib import Path
import subprocess
import sys
import numpy as np
import pandas as pd
from typing import List, Tuple, Dict, Any
import gzip
from datetime import datetime

from dotenv import load_dotenv
load_dotenv("../.env")

if os.environ.get('CDF_LIB', '') == '':
    print('No CDF_LIB environment variable found for CDF file processing.')
from spacepy import pycdf

In [3]:
URI = Path("srem.psi.ch/datarepo/V0/irem")

DATA_RAW = Path("../data_irem") / URI
DATA_EXTRACTED = Path("../data_irem/extracted/")
DATA_CSV = Path("../data_irem/csv/")

## Fetching data

run `0_fetch_data.py` to download the data from the SREM server.

## Extracting data

In [4]:
def get_data_raw_filenames():
    filenames = []
    for dirname in os.listdir(DATA_RAW):
        for filename in os.listdir(DATA_RAW / dirname):
            name = DATA_RAW / dirname / filename
            filenames.append(name)
    return sorted(filenames)

def extract_data_raw_filename(input_filename: Path, output_filename: Path):
    with open(input_filename, 'rb') as f:
        data = f.read()
        decompressed = gzip.decompress(data)
        with open(output_filename, 'wb') as g:
            return g.write(decompressed)
        
def extract_data_raw():
    filenames = get_data_raw_filenames()
    for filename in filenames:
        output_filename = DATA_EXTRACTED / filename.name[:-3]
        print(output_filename)
        extract_data_raw_filename(filename, output_filename)

In [5]:
extract_data_raw()

../data_irem/extracted/IREM_PACC_20021017.cdf
../data_irem/extracted/IREM_PACC_20021018.cdf
../data_irem/extracted/IREM_PACC_20021019.cdf
../data_irem/extracted/IREM_PACC_20021020.cdf
../data_irem/extracted/IREM_PACC_20021021.cdf
../data_irem/extracted/IREM_PACC_20021022.cdf
../data_irem/extracted/IREM_PACC_20021023.cdf
../data_irem/extracted/IREM_PACC_20021024.cdf
../data_irem/extracted/IREM_PACC_20021025.cdf
../data_irem/extracted/IREM_PACC_20021026.cdf
../data_irem/extracted/IREM_PACC_20021027.cdf
../data_irem/extracted/IREM_PACC_20021028.cdf
../data_irem/extracted/IREM_PACC_20021029.cdf
../data_irem/extracted/IREM_PACC_20021030.cdf
../data_irem/extracted/IREM_PACC_20021031.cdf
../data_irem/extracted/IREM_PACC_20021101.cdf
../data_irem/extracted/IREM_PACC_20021102.cdf
../data_irem/extracted/IREM_PACC_20021103.cdf
../data_irem/extracted/IREM_PACC_20021104.cdf
../data_irem/extracted/IREM_PACC_20021105.cdf
../data_irem/extracted/IREM_PACC_20021106.cdf
../data_irem/extracted/IREM_PACC_2

## CDF Processing

### Reading CDF data

Output: `pycdf.CDF`

In [6]:
def check_file_empty(filename: Path) -> bool:
    return os.stat(filename).st_size == 0

def read_cdf(cdf_path: Path) -> pycdf.CDF | None:
    """
    Note: It keeps the CDF file open, so it should be closed after use.
    """
    if check_file_empty(cdf_path):
        print(f"File {cdf_path} is empty.", file=sys.stderr)
        return None

    return pycdf.CDF(str(cdf_path))


def read_cdf_and_cache(cdf_path: Path)-> pycdf.CDF:
    cdf = None
    with pycdf.CDF(str(cdf_path)) as cdf:
        cdf = cdf.copy()
    return cdf

In [16]:
def is_cdf_filename_filtered(filename: str, datetime_filter: datetime) -> bool:
    # filename: IREM_PACC_YYYYMMDD.cdf
    date_str = filename[10:18]
    date = datetime.strptime(date_str, "%Y%m%d")
    return date >= datetime_filter

def read_irem_cdfs(cdf_path: Path, 
                   datetime_filter: datetime = datetime(1000, 1, 1)) -> List[pycdf.CDF]:
    cdfs = []
    for filename in sorted(os.listdir(cdf_path)):
        if filename.endswith(".cdf") \
            and is_cdf_filename_filtered(filename, datetime_filter):
            path = cdf_path / filename
            cdf = read_cdf(path)
            if cdf is not None:
                cdfs.append(cdf)
    return cdfs

### Exploring CDF data

In [17]:
def print_cdf_report(cdf: pycdf.CDF):
    print(f'Keys:')
    print(cdf)

    print(f'\nCDF meta:')
    print(cdf.meta)
    for key, val in cdf.items(): 
        print(f'\n{key} -> {val}')
        print(val.meta)

In [43]:
print_cdf_report(read_cdf(DATA_EXTRACTED / "IREM_PACC_20210101.cdf"))

Keys:
COUNTRATE: CDF_DOUBLE [14, 15]
ELECPARAM: CDF_DOUBLE [14, 3]
ELECPARERR: CDF_DOUBLE [14, 3]
EPOCH: CDF_EPOCH [14]
FITQUAL: CDF_INT2 [14]
LSHELL: CDF_DOUBLE [14]
MAGFIELD: CDF_DOUBLE [14, 3]
ORBIT: CDF_DOUBLE [14, 3]
PROTPARAM: CDF_DOUBLE [14, 3]
PROTPARERR: CDF_DOUBLE [14, 3]
label_COUNTERS: CDF_CHAR*3 [15] NRV
label_ELECPARAM: CDF_CHAR*40 [3] NRV
label_MAGFIELD: CDF_CHAR*7 [3] NRV
label_ORBIT: CDF_CHAR*6 [3] NRV
label_PROTPARAM: CDF_CHAR*40 [3] NRV

CDF meta:
Data_type: PACC>Processed Accumulation Data [CDF_CHAR]
Data_version: preliminary [CDF_CHAR]
Descriptor: SREM>Standard Radiation Environment Monitor [CDF_CHAR]
Discipline: Space Physics>Magnetospheric Science [CDF_CHAR]
Instrument_type: Particles (space) [CDF_CHAR]
Logical_file_id: IREM_PACC_20210101 [CDF_CHAR]
Logical_source: IREM_PACC [CDF_CHAR]
Logical_source_description: IREM Processed Accumulation Data [CDF_CHAR]
Mission_group: Integral [CDF_CHAR]
PI_affiliation: Paul Scherrer Institute, PSI [CDF_CHAR]
PI_name: W, Hajda

## CSV/DataFrame processing

### Converting CDF to DataFrame

In [18]:
def cdf_to_df(cdf: pycdf.CDF) -> pd.DataFrame:
    df = pd.DataFrame((cdf[key][...] for key in cdf.keys())).T
    df.columns = cdf.keys()
    return df

### Loading / Saving CSV

In [19]:
def load_csv(filename: str) -> pd.DataFrame:
    """
    Load a CSV file into a pandas DataFrame
    """

    df = pd.read_csv(filename)
    df['time'] = pd.to_datetime(df['time'])
    return df

def save_csv(df: pd.DataFrame, name: str) -> None:
    """
    Generate a name and save a pandas DataFrame to a CSV file
    """
    latest_time = max(df['time']).strftime('%Y%m%dT%H%M%S')
    filename = DATA_CSV.joinpath(name + "_" + latest_time + '.csv')

    df.to_csv(filename, index=False)

### Fixing DataFrame data

In [20]:
def fix_df_time(df: pd.DataFrame) -> pd.DataFrame:
    """
    Convert the time column to datetime and floor it to seconds, in place.
    """
    df["time"] = pd.to_datetime(df['time']).dt.floor('S')

    return df

# def fix_df_time_start(df: pd.DataFrame) -> pd.DataFrame:
#     """
#     Filter the dataframe to only include events after September 1, 2023, in place.
#     """
#     df.query("time >= '2023-09-01'", inplace=True)

#     return df

def fix_df_duplicates(df: pd.DataFrame) -> pd.DataFrame:
    """
    Find and remove duplicates from the dataframe, in place.
    """
    df.drop_duplicates(inplace=True, keep="first")
    return df

def fix_sorting_df(df: pd.DataFrame) -> pd.DataFrame:
    """
    Sort the dataframe by time, in place.
    """
    df.sort_values("time", inplace=True)
    return df

def fix_df(df: pd.DataFrame) -> pd.DataFrame:
    """
    Fix the dataframe in place.
    """
    fix_df_time(df)
    # fix_df_time_start(df)
    fix_df_duplicates(df)
    fix_sorting_df(df)
    return df

### Validating DataFrame data

In [21]:
def verify_df_sorted(df: pd.DataFrame) -> None:
    """
    Verify that the dataframe is sorted by "time"
    """
    # Find rows where the 'time' is decreasing from the previous row
    not_sorted_mask = df['time'].diff().dt.total_seconds() < 0

    # The first row can't be "not sorted" by definition, so we can exclude it from the mask
    not_sorted_mask.iloc[0] = False

    # Filter the DataFrame to find the not sorted rows
    not_sorted_rows = df[not_sorted_mask]

    if not df['time'].is_monotonic_increasing:
        raise ValueError(f"Dataframe is not sorted by time:\n{not_sorted_rows}")

def verify_df_time_diffs(df: pd.DataFrame, 
                         max_diff_tolerance: np.timedelta64 = np.timedelta64(90, 's'), 
                         min_diff_tolerance: np.timedelta64 = np.timedelta64(500, 'ms')) -> None:
    """
    Verify that the time differences between events are within tolerance.
    If time diff >= max_diff_tolerance, just prints the warning (data holes are permitted).
    If time diff <= min_diff_tolerance, raises an exception (possible floating point errors).
    
    Assumes that the dataframe is non-decreasingly sorted by "time".  
    
    There may me multiple groups of events with the same time.
    
    Args:
        df (pd.DataFrame): input dataframe with "time" column
        max_diff_tolerance (np.timedelta64, optional): max time difference tolerance in ms (warning only)
        min_diff_tolerance (np.timedelta64, optional): min time difference tolerance in ms (exception)

    Raises:
        ValueError: when time differences < min_diff_tolerance (possible floating point errors)
    """

    # get all unique "time" values in df
    times = df['time'].unique()

    # calc time diffs
    time_diffs = np.diff(times)

    # check if all time diffs are not larger than the tolerance
    checks = max_diff_tolerance > time_diffs
    if not all(checks):
        # find all indexes of unmet conditions
        indexes = np.where(checks == False)[0]

        # create a dataframe of times
        df_times = pd.DataFrame(times, columns=["time"])

        # find all holes
        holes = [f"{df_times.iloc[i]['time']} and {df_times.iloc[i + 1]['time']}" for i in indexes]
        
        print("Found time holes out of tolerance at times:", *holes, sep='\n\t')


    # check if all time diffs are not smaller than the tolerance
    # (possible floating point errors)
    checks = min_diff_tolerance < time_diffs
    if not all(checks):
        # find all indexes of unmet conditions
        indexes = np.where(checks == False)[0]

        # create a dataframe of times
        df_times = pd.DataFrame(times, columns=["time"])

        # find all too close values
        too_close = [f"{df_times.iloc[i]['time']} and {df_times.iloc[i + 1]['time']}" for i in indexes]
        
        raise ValueError(
            "Found time values too close to each other at times " +
            "(possible floating point errors):\n\t" +
            "\n\t".join(too_close))

# Procesing data categories

## Particle counters

In [22]:
def check_irem_cdf_labels_order(cdf: pycdf.CDF) -> bool:
    pattern = ['TC1', 'S12', 'S13', 'S14', 'S15', 'TC2', 'S25', 'C1 ', 'C2 ',
       'C3 ', 'C4 ', 'TC3', 'S32', 'S33', 'S34']
    return cdf["label_COUNTERS"][...].tolist() == pattern

def check_irem_cdfs(irem_cdfs: List[pycdf.CDF]) -> bool:
    for cdf in irem_cdfs:
        if not check_irem_cdf_labels_order(cdf):
            return False
    return True

def process_irem_particles(cdf: pycdf.CDF, scaler_start: int, scaler_end: int) -> pd.DataFrame:
    # According do the IREM User Manual:
    # CDF label_COUNTERS always are: 
    #   - [TC1 S12 S13 S14 S15 TC2 S25 C1 C2 C3 C4 TC3 S32 S33 S34]
    #   - and are 3 characters long e.g. "C1 "
    n = len(cdf["EPOCH"][...])
    times = cdf["EPOCH"][...]

    d_scalers = cdf["label_COUNTERS"][..., scaler_start:scaler_end]
    d = cdf["COUNTRATE"][..., scaler_start:scaler_end]

    time_col = np.repeat(times, len(d_scalers))
    scaler_col = np.tile(d_scalers, n)
    value_col = d.flatten()
    bin_col = np.tile(np.arange(1, 1 + len(d_scalers)), n)

    df = pd.DataFrame({
        "time": time_col,
        "scaler": scaler_col,
        "value": value_col,
        "bin": bin_col
    })

    return df

def process_irem_d1(cdf: pycdf.CDF) -> pd.DataFrame:
    # According do the IREM User Manual:
    # label_COUNTERS[0:5] is [TC1 S12 S13 S14 S15] which is D1
    return process_irem_particles(cdf, 0, 5)

def process_irem_d2(cdf: pycdf.CDF) -> pd.DataFrame:
    # According do the IREM User Manual:
    # label_COUNTERS[5:7] is [TC2 S25] which is D2
    return process_irem_particles(cdf, 5, 7)

def process_irem_coincidence(cdf: pycdf.CDF) -> pd.DataFrame:
    # According do the IREM User Manual:
    # label_COUNTERS[7:11] is [C1 C2 C3 C4] which is D1+D2 Coincidence
    return process_irem_particles(cdf, 7, 11)

def process_irem_d3(cdf: pycdf.CDF) -> pd.DataFrame:
    # According do the IREM User Manual:
    # label_COUNTERS[11:15] is [TC3 S32 S33 S34] which is D3
    return process_irem_particles(cdf, 11, 15)

In [36]:
# WARNING THIS CELL CAN CONSULE ~20GB OF RAM with no filter

# 1. Reading CDFs
DATETIME_FILTER = datetime(2023, 9, 1)

irem_cdfs = read_irem_cdfs(DATA_EXTRACTED, datetime_filter=DATETIME_FILTER)
print("Found irem CDFs:", len(irem_cdfs))
irem_cdf_check = check_irem_cdfs(irem_cdfs)
print(f"Checking irem CDFs: {irem_cdf_check}")
if not irem_cdf_check:
    raise ValueError("irem CDFs check failed.")

Found irem CDFs: 346
Checking irem CDFs: True


In [37]:
def pipeline(cdfs: List[pycdf.CDF], process_fn, name) -> None:
    # WARNING: CAN CONSUME ~15GB OF RAM!!!!

    # 2. Converting CDFs to DataFrames
    print("Processing")
    df = pd.concat((
        process_fn(cdf) for cdf in cdfs
    ))
    print("measurements:", len(df))

    # 3. Fixing DataFrames
    fix_df(df)
    print("measurements:", len(df), "(after fixing)")

    # 4. Verifying DataFrames
    # ..............

    # 5. Saving DataFrames to CSV
    print("Saving CSVs")
    save_csv(df, name)

    # 6. Clueaning up RAM
    del df

In [38]:
# WARNING: CAN CONSUME ~15GB OF RAM!!!!
pipeline(irem_cdfs, process_irem_d1, "irem_d1")

Processing
measurements: 2382735


/tmp/ipykernel_117287/1021181258.py:5: FutureWarning: 'S' is deprecated and will be removed in a future version, please use 's' instead.
  df["time"] = pd.to_datetime(df['time']).dt.floor('S')


measurements: 2382735 (after fixing)
Saving CSVs


In [39]:
# WARNING: CAN CONSUME ~15GB OF RAM!!!!
pipeline(irem_cdfs, process_irem_d2, "irem_d2")

Processing
measurements: 953094
measurements: 953094 (after fixing)
Saving CSVs


/tmp/ipykernel_117287/1021181258.py:5: FutureWarning: 'S' is deprecated and will be removed in a future version, please use 's' instead.
  df["time"] = pd.to_datetime(df['time']).dt.floor('S')


In [40]:
# WARNING: CAN CONSUME ~15GB OF RAM!!!!
pipeline(irem_cdfs, process_irem_d3, "irem_d3")

Processing
measurements: 1906188


/tmp/ipykernel_117287/1021181258.py:5: FutureWarning: 'S' is deprecated and will be removed in a future version, please use 's' instead.
  df["time"] = pd.to_datetime(df['time']).dt.floor('S')


measurements: 1906188 (after fixing)
Saving CSVs


In [41]:
# WARNING: CAN CONSUME ~15GB OF RAM!!!!
pipeline(irem_cdfs, process_irem_coincidence, "irem_coin")

Processing
measurements: 1906188


/tmp/ipykernel_117287/1021181258.py:5: FutureWarning: 'S' is deprecated and will be removed in a future version, please use 's' instead.
  df["time"] = pd.to_datetime(df['time']).dt.floor('S')


measurements: 1906188 (after fixing)
Saving CSVs


In [42]:
del irem_cdfs